In [4]:
import os
import json
tokens=""
docs=[]

for split in ["val"]:#"train",'test',"val"]:
    path = "/home/xueweiwa/Capstone/xueweiwa/data/preprocess/"+split
    
    for fname in os.listdir(path):
            if fname[0]=='.':
                continue
                
            data = json.load(open(os.path.join(path,fname),'r'))
            doc = " ".join(data['article'])
            doc = doc.lower()
            tokens += doc + " "
            docs.append(doc)


KeyboardInterrupt: 

In [9]:
len(tokens)

3201703

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
  



stopwords = set(STOPWORDS) 
stopwords.update(set(["s","lrb","rrb","one"]))

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(tokens) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [5]:
import os
import json
docs=[]

for split in ["val"]:#"train",'test',"val"]:
    path = "/home/xueweiwa/Capstone/xueweiwa/data/preprocess/"+split
    
    for fname in os.listdir(path):
            data = json.load(open(os.path.join(path,fname),'r'))
            doc = " ".join(data['article'])
            doc = doc.lower()
            docs.append(doc)
            


In [6]:
from gensim import corpora
import os
import random
import codecs
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
 
# Function to remove stop words from sentences & lemmatize words.
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y


stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
doc_clean = [clean(doc) for doc in docs]


# Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(doc_clean)
 
# Filter terms which occurs in less than 4 articles & more than 40% of the articles
dictionary.filter_extremes(no_below=4, no_above=0.4)
 
# List of few words which are removed from dictionary as they are content neutral
stoplist = set('also use make people know many call include part find become like mean often different \
               usually take wikt come give well get since type list say change see refer actually iii \
               aisne kinds pas ask would way something need things want every str -lsb- -rsb-'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [7]:
import nltk
# nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/xueweiwa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


In [14]:
from gensim.models.ldamodel import LdaModel as Lda
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=8, id2word = dictionary, passes=30, iterations=500)
 
# dump LDA model using cPickle for future use
ldafile = open('lda_model_sym_wiki.pkl','wb')
# cPickle.dump(ldamodel,ldafile)
# ldafile.close()
 


In [13]:
len(doc_term_matrix)

9725

In [10]:
import pickle
pickle.dump(ldamodel,ldafile)
ldafile.close()


In [18]:
# Print all the 50 topics
for i,topic in enumerate(ldamodel.print_topics(num_topics=100, num_words=8)):
    words = topic[1].split("+")
    print(words)
    print("=====")

['0.014*"war" ', ' 0.008*"force" ', ' 0.006*"army" ', ' 0.005*"british" ', ' 0.004*"military" ', ' 0.004*"german" ', ' 0.003*"battle" ', ' 0.003*"attack"']
=====
['0.006*"write" ', ' 0.006*"century" ', ' 0.005*"work" ', ' 0.004*"name" ', ' 0.004*"church" ', ' 0.004*"form" ', ' 0.003*"language" ', ' 0.003*"early"']
=====
['0.020*"season" ', ' 0.019*"game" ', ' 0.017*"team" ', ' 0.011*"play" ', ' 0.010*"league" ', ' 0.008*"match" ', ' 0.008*"club" ', ' 0.008*"score"']
=====
['0.018*"film" ', ' 0.010*"release" ', ' 0.007*"series" ', ' 0.006*"life" ', ' 0.006*"show" ', ' 0.005*"music" ', ' 0.005*"write" ', ' 0.004*"album"']
=====
['0.006*"may" ', ' 0.005*"system" ', ' 0.004*"process" ', ' 0.004*"model" ', ' 0.003*"form" ', ' 0.003*"cause" ', ' 0.003*"formula" ', ' 0.003*"number"']
=====
['0.010*"city" ', ' 0.006*"south" ', ' 0.006*"north" ', ' 0.005*"area" ', ' 0.005*"population" ', ' 0.004*"build" ', ' 0.004*"century" ', ' 0.004*"west"']
=====
['0.007*"government" ', ' 0.006*"unite" ', ' 